In [1]:
import pandas as pd
import numpy as np
import os
import scipy.stats as stats
from utils import load_data, val_loglhood

# Gibbs Sampling

In [2]:
def gibbs_sampling(iters, data_path, K, p, q, debug=False):
    """
    data_path: path where data is saved.
    K: number of plants (n_plants in load_data function).
    p: time interval to consider.
    q: sample distribution for parameters.
    """
    if debug:
        print('Loading data...')
    Y0, X = load_data(data_path, K, p, resample_rule='4H')
    if debug:
        print('Y0 shape: {}'.format(Y0.shape))
        print('X shape: {}'.format(X.shape))
    theta = init_parameters(K, p, q, Y0, X, debug)
    if debug:
        print('Parameters intialized!')
    samples = []
    for i in range(iters):
        if debug:
            print('Iteration {}'.format(i))
        for j in theta.shape[0]:
            while True:
                theta_j = q.rvs()
                lk = val_loglhood(theta, Y0, X, True)
                if lk != -np.inf:
                    break
        samples.append(theta)
    return np.array(samples)
        
    
def init_parameters(K, p, q, Y0, X, debug=False):
    if debug:
        print('Initializing parameters...')
    while True:
        theta = np.zeros(K ** 2 * (p + 1))
        for i in range(theta.shape[0]):
            theta[i] = q.rvs()

        # Force CovU to be positive semidefinite.
        covu = np.reshape(theta[-K**2:], (K, K)).T
        covu = np.dot(covu.T, covu)
        theta[-K**2:] = np.reshape(covu, K**2)
        
        lk = val_loglhood(theta, Y0, X, True)
        print('LK = {}'.format(lk))
        if lk != -np.inf:
            return theta
        

In [3]:
a = np.arange(20)
print(a)
b = a[-3 ** 2:]
print(b)
c = np.reshape(b, (3, 3)).T
print(c)
print(np.dot(c.T, c))
d = np.reshape(b, 3 ** 2)
print(d)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]
[11 12 13 14 15 16 17 18 19]
[[11 14 17]
 [12 15 18]
 [13 16 19]]
[[434 542 650]
 [542 677 812]
 [650 812 974]]
[11 12 13 14 15 16 17 18 19]


In [6]:
DATA_PATH = '/home/chrisams/Documents/datasets/data_TAIM/processed/'
q = stats.norm(loc=0, scale=1)
K = 2
p = 2
iters = 5
debug = True

In [7]:
gibbs_sampling(iters, DATA_PATH, K, p, q, debug)

Loading data...
(2, 16611)
Y0 shape: (2, 16609)
X shape: (4, 16609)
Initializing parameters...
Matriz A no es estable
[[-0.76250507  0.85474691  0.84335295  0.60784817]
 [ 0.18331089 -0.04156839  2.02153957 -0.8417788 ]]
[1.2151764  1.19020606 1.15777778 1.15777778]
LK = -inf
Matriz A no es estable
[[ 0.18990287 -0.78127938  1.46574304  0.73098463]
 [-1.38455897 -0.55838621 -0.48110793 -0.90547731]]
[1.40562967 1.58013752 0.66272595 0.66272595]
LK = -inf
Matriz A no es estable
[[-0.34157921  0.65721047 -0.02191091 -1.03035704]
 [-0.04871463  0.95593519  1.37448484  1.07304696]]
[1.6254676  0.6363917  1.16031839 1.16031839]
LK = -inf
Matriz A no es estable
[[-0.13689042  0.23370372  0.10689688 -1.01212963]
 [ 1.17148186  0.89095594  0.07992095 -0.6345136 ]]
[1.24657168 1.24657168 0.86731934 0.00969223]
LK = -inf
Matriz A no es estable
[[-0.63917831  1.45495724  0.61700463 -1.52652196]
 [-0.32338408 -0.24339119  1.09249311 -0.6789114 ]]
[1.57434324 1.57434324 0.70982463 0.70982463]
LK = 

TypeError: 'int' object is not iterable